# Multi-cycle image registration

It is often advantageous to acquire data in multiple cycles, in which different objects are stained in different imaging cycles. A key step in preprocessing these kinds of data is image registration. This is implemented in ``blimp`` using functions in ``blimp.preprocessing.registration``. This notebook demonstrates the use of these functions for correcting image data.

Note: You must run ``notebooks/0_setup.ipynb`` first to specify the configuration and download the test data.

In [5]:
import napari

from IPython.display import display
import numpy as np
import matplotlib.pyplot as plt
import logging
import skimage

from blimp.data import load_example_data
from blimp.log import configure_logging
from blimp.constants import blimp_config
from blimp.preprocessing.registration import (
    calculate_shifts,
    apply_shifts,
    register_2D,
    transform_2D,
    TransformationParameters
)
from aicsimageio import AICSImage
from pathlib import Path

configure_logging(verbosity=2)
# ensure that example data is downloaded
load_example_data()
# read correct blimp_config -- created with 0_setup.ipynb
blimp_config.config_fname = "blimp.ini"
print(blimp_config)


unpacked_dir = /Users/scottberry/source/blimp/notebooks/_data/raw
archive_path = /Users/scottberry/source/blimp/notebooks/_data/archive/_data.zip
Reading config from blimp.ini
BLIMPConfig (fname: blimp.ini)
EXPERIMENT_DIR: /Users/scottberry/source/blimp/notebooks/_experiments
BASE_DATA_DIR: /Users/scottberry/source/blimp/notebooks/_data/raw
data_config/exampledata: /Users/scottberry/source/blimp/notebooks/ExampleData_constants.py



Load some images from the examples directory using the ``aicsimageio`` package

In [6]:
cyc01 = AICSImage(Path(blimp_config.BASE_DATA_DIR) / '_data' / 'operetta_cls_multiplex' / 'cycle_01' / 'r05c03f15-fk1fl1-mip.ome.tiff')
cyc02 = AICSImage(Path(blimp_config.BASE_DATA_DIR) / '_data' / 'operetta_cls_multiplex' / 'cycle_02' / 'r05c03f15-fk1fl1-mip.ome.tiff')

The default XML parser will be changing from 'xmlschema' to 'lxml' in version 0.4.0.  To silence this warning, please provide the `parser` argument, specifying either 'lxml' (to opt into the new behavior), or'xmlschema' (to retain the old behavior).


In [7]:
print(cyc01.channel_names)
print(cyc01.dims)

['DAPI', 'Alexa 568']
<Dimensions [T: 1, C: 2, Z: 1, Y: 2160, X: 2160]>


get some input data

In [8]:
dapi01 = cyc01.get_image_data("YX",C=0)
dapi02 = cyc02.get_image_data("YX",C=0)

generate a rotated version of dapi02 and crop dapi01 to match size

In [9]:
dapi02_rotated = skimage.transform.rotate(dapi02,3,preserve_range=True).astype(np.uint16)
# remove outer 100 pixels
dapi02_rotated_crop = dapi02_rotated[100:-100,100:-100]
dapi01_crop = dapi01[100:-100,100:-100]

# multi-cycle data

Data acquired on the operetta in a real two-cycle experiment. 

In [10]:
viewer0 = napari.view_image(dapi01,colormap="red",blending="additive",name="Cycle 1 - uncorrected")
viewer0.add_image(dapi02,colormap="green",blending="additive",name="Cycle 2 - uncorrected")

<Image layer 'Cycle 2 - uncorrected' at 0x7fefa3852cd0>

In [11]:
print(register_2D.__doc__)

Align an image to a reference image, keeping transformation parameters.

    Aligns a ``moving`` image to a ``fixed`` image using ITK Elastix.

    Parameters
    ----------
    fixed
        a reference image
    moving
        an image to be aligned to the reference image
    settings
        object of class TransformationParameters to provide
        the initial setting for performing the registration
        See ``blimp.utils.TransformationParameters`` for
        more details.

    Returns
    -------
    numpy.ndarray
        registered image derived by transforming ``moving``
        to the reference frame of ``fixed``.
    TransformationParameters
        object containing the full transformation parameters
        necessary to perform the transformation of ``moving``
        to the reference frame of ``fixed``.

    Raises
    ------
    TypeError
        If any of the positional inputs are not of the correct type
        If the numpy.dtypes of the two input arrays do not matc

Align cycle 2 to cycle 1. The ``register_2D`` function uses the [``elastix`` library from ITK](https://github.com/InsightSoftwareConsortium/ITKElastix). More details on specifying transformation parameters is provided in the [Elastix wiki](https://github.com/SuperElastix/elastix/wiki).

In [12]:
settings = TransformationParameters('rigid')
dapi02_corrected, parameters_cyc01_cyc02 = register_2D(fixed=dapi01, moving=dapi02, settings=settings)

In [13]:
viewer0.add_image(dapi02_corrected,colormap="blue",blending="additive",name="Cycle 2 - corrected")

<Image layer 'Cycle 2 - corrected' at 0x7fef7045faf0>

## simulated rotations

Try with the simulated data that has been rotated

In [14]:
viewer1 = napari.view_image(dapi01_crop,colormap="red",blending="additive",name="Cycle 1 - original")
viewer1.add_image(dapi02_rotated_crop,colormap="green",blending="additive",name="Cycle 2 - rotated")

<Image layer 'Cycle 2 - rotated' at 0x7fef884cd160>

Test first using translation only

In [15]:
dapi02_rotated_crop_registered, translation_parameters = register_2D(
    fixed=dapi01_crop,
    moving=dapi02_rotated_crop,
    settings=TransformationParameters(transformation_mode='translation')
)

viewer1.add_image(dapi02_rotated_crop_registered,colormap="blue",blending="additive",name="Cycle 1 - registered (translation)")

<Image layer 'Cycle 1 - registered (translation)' at 0x7fef6153cac0>

now allow rotations in addition to translation

In [16]:
dapi02_rotated_crop_registered_rigid, rigid_parameters = register_2D(
    fixed=dapi01_crop,
    moving=dapi02_rotated_crop,
    settings=TransformationParameters(transformation_mode='rigid')
)

viewer1.add_image(dapi02_rotated_crop_registered_rigid,colormap="blue",blending="additive",name="Cycle 1 - registered (rotation)")

<Image layer 'Cycle 1 - registered (rotation)' at 0x7fef94c8da00>

# Saving / loading transformation parameters

To align multiple cycles with one another, we need to apply the transformation settings from the reference channel to the other channels.

In [17]:
import os
results_path = Path(blimp_config.EXPERIMENT_DIR) / "registration"
if not results_path.exists():
    os.makedirs(results_path)

Save the parameters aligning cycle 1 to cycle 2

In [18]:
parameters_cyc01_cyc02.save(results_path / "parameters_cyc01_cyc02.txt")

These can be re-loaded using the ``TransformationParameters`` class with ``from_file``

In [19]:
parameters_cyc01_cyc02_loaded = TransformationParameters(from_file=results_path / 'parameters_cyc01_cyc02.txt')
parameters_cyc01_cyc02_loaded

Now align all channels using these parameters. Note the use of the ``transform_2D`` function, which uses pre-defined parameters.

In [20]:
cyc02_arrays = [cyc02.get_image_dask_data('YX',C=c) for c in range(cyc02.dims.C)]
cyc02_registered = [transform_2D(moving=arr, parameters=parameters_cyc01_cyc02_loaded) for arr in cyc02_arrays]

and visualize the full multi-channel image

In [21]:
viewer2 = napari.Viewer()
viewer2.add_image(dapi01,colormap="blue",blending="additive",name="Cycle 1 - channel 0")
viewer2.add_image(cyc01.get_image_dask_data('YX',C=1),colormap="green",blending="additive",name="Cycle 1 - channel 1")
viewer2.add_image(cyc02_registered[0],colormap="blue",blending="additive",name="Cycle 2 - channel 0")
viewer2.add_image(cyc02_registered[1],colormap="green",blending="additive",name="Cycle 2 - channel 1")
viewer2.add_image(cyc02_registered[2],colormap="red",blending="additive",name="Cycle 2 - channel 2")

<Image layer 'Cycle 2 - channel 2' at 0x7fefb375adc0>

# Register a list of images

As in the above example, we often want to register a set of images together, using a common channel captured in all images for alignment. This becomes cumbersome using the above framework, but is readily achieved with the higher-level functions, ``calculate_shifts`` and ``apply_shifts``. 

In [22]:
# defaults to elastix-based registration with 'rigid' transformation
parameters_list = calculate_shifts(
    images=[cyc01,cyc02],
    reference_channel=0,
    reference_cycle=0
)

2023-01-30 22:08:28 | INFO     | Using ``elastix`` library to align 2 x 2D images. [[calculate_shifts @ /Users/scottberry/source/blimp/blimp/preprocessing/registration.py:546]]


2023-01-30 22:08:28 | WARNING  | 
                Using default 'rigid' transformation to align images.
                Specify alternative settings using the ``registration_settings`` argument.
                 [[calculate_shifts @ /Users/scottberry/source/blimp/blimp/preprocessing/registration.py:554]]


In [23]:
print(parameters_list[1])

ParameterObject (0x7fefd7bd2af0)
  RTTI typeinfo:   elastix::ParameterObject
  Reference Count: 1
  Modified Time: 183875
  Debug: Off
  Object Name: 
  Observers: 
    none
ParameterMap 0: 
  (CenterOfRotationPoint 1079.5 1079.5)
  (CompressResultImage "false")
  (DefaultPixelValue 0)
  (Direction 1 0 0 1)
  (FinalBSplineInterpolationOrder 3)
  (FixedImageDimension 2)
  (FixedInternalImagePixelType "float")
  (HowToCombineTransforms "Compose")
  (Index 0 0)
  (InitialTransformParametersFileName "NoInitialTransform")
  (MovingImageDimension 2)
  (MovingInternalImagePixelType "float")
  (NumberOfParameters 3)
  (Origin 0 0)
  (ResampleInterpolator "FinalBSplineInterpolator")
  (Resampler "DefaultResampler")
  (ResultImageFormat "nii")
  (ResultImagePixelType "float")
  (Size 2160 2160)
  (Spacing 1 1)
  (Transform "EulerTransform")
  (TransformParameters 6.2116e-05 -61.9557 -3.57572)
  (UseDirectionCosines "true")



These can be applied using the ``apply_shifts`` function

In [24]:
registered_images = apply_shifts([cyc01,cyc02],parameters_list)

Check the results

In [25]:
viewer3 = napari.Viewer()
for cycle in range(2):
    viewer3.add_image(
        registered_images[cycle].get_image_dask_data('CYX'),
        name=[str(cycle) + '_' + name for name in registered_images[cycle].channel_names],
        channel_axis=0,
        blending='additive')


``apply_shifts`` has a ``crop`` argument, which is ``False`` by default. ``crop=True`` ensures all images are cropped to the same size, equal to the maximum rectangle found in all images.

In [26]:
registered_images = apply_shifts([cyc01,cyc02],parameters_list,crop=True)
viewer4 = napari.Viewer()
for cycle in range(2):
    viewer4.add_image(
        registered_images[cycle].get_image_dask_data('CYX'),
        name=[str(cycle) + '_' + name for name in registered_images[cycle].channel_names],
        channel_axis=0,
        blending='additive')


# FFT-based registration

An alternative to ``elastix`` is the [``image_registration``](https://github.com/keflavich/image_registration) package, which is intended for image registration where the brightness is “extended” or “spread out” . An interface to this is provided also using ``register_2D_fast``, and ``register_images``/``align_images`` with ``lib='image_registration'``. Functionality is limited to x-y translations and differs from ``elastix`` in that interpolation is not performed. Images are merely shifted by integer values.

In [27]:
from blimp.preprocessing.registration import register_2D_fast
dapi02_aligned_fast, parameters_fast = register_2D_fast(fixed=dapi01,moving=dapi02)

Results can be viewed with the other alignments

In [28]:
viewer0.add_image(dapi02_aligned_fast,colormap="green",blending="additive",name="Cycle 2 - corrected (fast)")

<Image layer 'Cycle 2 - corrected (fast)' at 0x7fef898f3400>

In [29]:
parameters_list = calculate_shifts(
    images=[cyc01,cyc02],
    reference_channel=0,
    reference_cycle=0,
    lib='image_registration'
)

2023-01-30 22:08:51 | INFO     | Using ``image_registration`` library to align 2 x 2D images. [[calculate_shifts @ /Users/scottberry/source/blimp/blimp/preprocessing/registration.py:568]]


In [30]:
parameters_list

[(0, 0), (-62, -3)]

In [31]:
registered_images_fast = apply_shifts([cyc01,cyc02],parameters_list,'image_registration')

In [32]:
viewer5 = napari.Viewer()
for cycle in range(2):
    viewer5.add_image(
        registered_images_fast[cycle].get_image_dask_data('CYX'),
        name=[str(cycle) + '_' + name for name in registered_images_fast[cycle].channel_names],
        channel_axis=0,
        blending='additive')